Aquí instanciamos el conector singleton a la base de datos y verificamos que la sesión se haya abierto correctamente.

In [1]:
from src.database.db_connector import DatabaseConnector

db = DatabaseConnector()
session = db.get_session()
print("✅ Conexión exitosa")


✅ Conexión exitosa


### Consulta de las primeras 5 filas de ventas
Consultamos los primeros registros de la tabla `sales` para verificar que la conexión y los datos estén cargados correctamente.

In [2]:
df = db.run_query("SELECT * FROM sales LIMIT 5")
df.head()

,sale_id,employee_id,customer_id,product_id,quantity,discount,total_price,sale_time,transaction_number
0,14,7,33133,406,9,0.00,63.00,39:52.0,VOP9A7Y4C5XSM2LLT0UJ\r
1,170,10,19039,377,5,0.00,50.00,33:40.0,VIIKLFPVD2D5U5E3NKKA\r
2,416,16,71212,351,19,0.00,304.00,40:57.4,QFA3F3LANPC6UVQBPYL2\r
3,486,23,93253,417,24,0.00,552.00,59:29.4,6H8SDUI9MCA1IXZ4UCUE\r
4,659,13,65151,147,17,0.20,221.00,27:58.9,1O062RUFV0LVAQ94G38J\r


###  Uso del patrón Factory
Se instancia un objeto `Customer` a partir de una fila tipo diccionario. Esto permite desacoplar la construcción de objetos del origen de datos.

In [3]:
from src.factories.model_factory import ModelFactory

row = {
    "CustomerID": 1,
    "FirstName": "Matias",
    "MiddleInitial": "F",
    "LastName": "Zalazar",
    "CityID": 100,
    "Address": "Belgrano 111"
}

customer = ModelFactory.create_customer(row)
print(customer.__dict__)


{'_customer_id': 1, '_first_name': 'Matias', '_middle_initial': 'F', '_last_name': 'Zalazar', '_city_id': 100, '_address': 'Belgrano 111'}


### Ejecución de pruebas unitarias
Se corre `pytest` sobre la carpeta `test/` para validar la integridad de las clases, conexión y factories implementadas.

In [1]:
!pytest -q test/ --tb=short

...........                                                              [100%]
11 passed in 1.00s


### Registro de la ejecución
Marcamos la fecha y hora de finalización del proceso como evidencia de ejecución exitosa.

In [ ]:
from datetime import datetime
print("📦 Integración completada con éxito")
print("Fecha de ejecución:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

📦 Integración completada con éxito
Fecha de ejecución: 2025-06-04 21:40:15


## Consulta avanzada 
Esta consulta utiliza una Common Table Expression (CTE) llamada ventas_por_ciudad, donde se agrupan las ventas totales (SUM(total_price)) por ciudad. Luego, la consulta principal ordena los resultados de mayor a menor volumen de ventas.

Objetivo: Identificar las ciudades con mayores ventas totales.

In [4]:
query_cte = """WITH ventas_por_ciudad AS (
  SELECT c.city_id, ci.city_name, SUM(s.total_price) AS total_ventas
  FROM sales s
  JOIN customers c ON s.customer_id = c.customer_id
  JOIN cities ci ON c.city_id = ci.city_id
  GROUP BY c.city_id, ci.city_name
)
SELECT * FROM ventas_por_ciudad ORDER BY total_ventas DESC;
"""
df_cte = db.run_query(query_cte)
print(df_cte.head())

   city_id       city_name total_ventas
0       94         Yonkers    102038.00
1       81      Sacramento     93261.00
2       85  St. Petersburg     92329.00
3       54     Albuquerque     92055.00
4       71         Phoenix     90623.00


Esta consulta aplica la función de ventana RANK() para asignar un ranking a cada empleado, basado en el total vendido (SUM(total_price)). Se utiliza OVER(ORDER BY ...) para calcular el ranking de forma descendente.

Objetivo: Determinar el top 10 de empleados que generaron más ingresos.

In [5]:
query_rank = """SELECT 
  e.employee_id,
  e.first_name,
  SUM(s.total_price) AS total_vendido,
  RANK() OVER (ORDER BY SUM(s.total_price) DESC) AS ranking
FROM sales s
JOIN employees e ON s.employee_id = e.employee_id
GROUP BY e.employee_id, e.first_name
LIMIT 10;
"""
df_rank = db.run_query(query_rank)
print(df_rank.head())

   employee_id first_name total_vendido  ranking
0           23      Janet     665091.00        1
1           22      Tonia     631158.00        2
2           21      Devon     618744.00        3
3           20     Shelby     569220.00        4
4           19    Bernard     538954.00        5


En esta consulta se utiliza la función de ventana RANK() para clasificar a los clientes según el total gastado en compras. Se combina CONCAT() para mostrar el nombre completo y se agrupan las ventas por cliente.

Objetivo: Obtener el top 10 de clientes que más dinero gastaron, ordenados por total_gastado.

In [6]:
query_sales_per_customer = """SELECT 
  c.customer_id,
  CONCAT(c.first_name, ' ', c.last_name) AS cliente,
  SUM(s.total_price) AS total_gastado,
  RANK() OVER (ORDER BY SUM(s.total_price) DESC) AS posicion
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
LIMIT 10;
"""
df_sales_per_customer = db.run_query(query_sales_per_customer)
print(df_sales_per_customer.head())

   customer_id         cliente total_gastado  posicion
0        91038   Darcy Bullock       1872.00         1
1        94115    Blake Dalton       1848.00         2
2        96485  Forrest Morton       1700.00         3
3        98273   Curtis Harmon       1650.00         4
4        73360  Allison Davies       1615.00         5


Esta consulta utiliza una Common Table Expression (CTE) para calcular la cantidad total vendida de cada producto, agrupada por categoría. Luego, se aplica la función de ventana `RANK()` con `PARTITION BY` para generar un ranking dentro de cada categoría, permitiendo identificar los productos más vendidos por tipo.

Objetivo: Obtener el top 3 de productos más vendidos dentro de cada categoría, en función de la cantidad de unidades vendidas.


In [3]:
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

query_top_products = """
WITH producto_categoria AS (
  SELECT 
    p.product_id,
    p.product_name,
    p.category_id,
    SUM(s.quantity) AS total_vendido,
    RANK() OVER (PARTITION BY p.category_id ORDER BY SUM(s.quantity) DESC) AS posicion
  FROM sales s
  JOIN products p ON s.product_id = p.product_id
  GROUP BY p.product_id, p.product_name, p.category_id
)
SELECT * FROM producto_categoria
WHERE posicion <= 3;
"""

df_top_products = db.run_query(query_top_products)
print(df_top_products[['product_id', 'product_name', 'category_id', 'total_vendido', 'posicion']].head(10))

   product_id                     product_name  category_id total_vendido  posicion
0         156                Sprouts - Alfalfa            1          1910         1
1         250    Soup - Campbells; Beef Barley            1          1858         2
2         451  Soup - Campbells Tomato Ravioli            1          1728         3
3         383      Cake - Box Window 10x10x2.5            2          1882         1
4         212             Curry Paste - Madras            2          1811         2
5         418             Truffle Cups - Brown            2          1671         3
6          65                     Brandy - Bar            3          1904         1
7         177        Coconut - Shredded; Sweet            3          1896         2
8         424               Vinegar - Tarragon            3          1830         3
9         252     Nantucket - Pomegranate Pear            4          1657         1


Esta consulta utiliza una CTE llamada `ventas_rank` para identificar los días con mayor volumen de ventas. Se agrupan los datos por fecha (`DATE(s.sale_time)`) y se calcula el total vendido por día (`SUM(total_price)`). Luego, se utiliza la función de ventana `ROW_NUMBER()` para asignar una posición en el ranking de días más fuertes en términos de facturación.

Se incluye un filtro `WHERE DATE(s.sale_time) IS NOT NULL` para evitar registros con fechas nulas que puedan aparecer como `None` en el resultado.

Objetivo: Detectar los 10 días con mayor facturación total en el histórico de ventas.



In [16]:
query_dias_fuertes = """
WITH ventas_rank AS (
  SELECT
    DATE(s.sale_time) AS fecha,
    SUM(s.total_price) AS total_dia,
    ROW_NUMBER() OVER (ORDER BY SUM(s.total_price) DESC) AS dia_mas_fuerte
  FROM sales s
  WHERE DATE(s.sale_time) IS NOT NULL
  GROUP BY DATE(s.sale_time)
)
SELECT * FROM ventas_rank
LIMIT 10;
"""

df_dias_fuertes = db.run_query(query_dias_fuertes)
print(df_dias_fuertes)


        fecha total_dia  dia_mas_fuerte
0  2057-02-04   1762.00               1
1  2058-07-08   1718.00               2
2  2015-01-03   1514.00               3
3  2012-03-04   1503.00               4
4  2004-10-02   1463.00               5
5  2049-07-03   1447.00               6
6  2023-04-06   1434.00               7
7  2034-03-08   1380.00               8
8  2043-11-05   1361.00               9
9  2045-12-02   1357.00              10


Estas consultas permiten responder preguntas clave como:
- ¿Qué ciudades generan más ingresos?
- ¿Quiénes son los empleados más efectivos en ventas?
- ¿Qué clientes representan el mayor valor comercial?
Esta información puede ser utilizada para diseñar campañas, establecer prioridades de expansión o premiar a personal destacado.


## Vista SQL 

Creamos una vista que consolida información de ventas incluyendo datos del producto, cliente y empleado. Esto permite consultar fácilmente reportes enriquecidos sin escribir joins complejos cada vez.

Esta vista es útil para dashboards, análisis de ventas por empleados o revisión de tickets.
Consolida datos de ventas uniendo productos, clientes y empleados, lo que permite análisis más completos y reutilizables.

Objetivo del objeto SQL: facilitar la consulta reutilizable de datos.


In [7]:
from sqlalchemy import text

vista_sql = """
CREATE OR REPLACE VIEW vista_ventas_enriquecidas AS
SELECT 
    s.sale_id,
    s.total_price,
    s.discount,
    s.sale_time,
    p.product_name,
    c.first_name AS cliente,
    e.first_name AS empleado
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
JOIN employees e ON s.employee_id = e.employee_id
JOIN products p ON s.product_id = p.product_id
"""

db.get_session().execute(text(vista_sql))
print("✅ Vista creada correctamente")


✅ Vista creada correctamente


In [8]:

df_vista = db.run_query("SELECT * FROM vista_ventas_enriquecidas LIMIT 5")
df_vista

,sale_id,total_price,discount,sale_time,product_name,cliente,empleado
0,6950,119.00,0.00,46:33.6,Flour - Whole Wheat,Toby,Seth
1,281342,294.00,0.20,04:25.2,Flour - Whole Wheat,Judith,Devon
2,299898,187.00,0.00,48:48.1,Flour - Whole Wheat,Marco,Sonya
3,460059,345.00,0.00,16:34.1,Flour - Whole Wheat,Marla,Janet
4,521261,68.00,0.00,02:27.6,Flour - Whole Wheat,Shari,Darnell


### Vista: Clientes Frecuentes
Esta vista permite identificar los clientes que realizaron más compras y cuánto gastaron en total. Ideal para segmentar usuarios recurrentes o aplicar estrategias de fidelización.

In [17]:
from sqlalchemy import text

vista_clientes_frecuentes = """
CREATE OR REPLACE VIEW vista_clientes_frecuentes AS
SELECT 
    c.customer_id,
    CONCAT(c.first_name, ' ', c.last_name) AS cliente,
    COUNT(s.sale_id) AS cantidad_compras,
    SUM(s.total_price) AS total_gastado
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name
ORDER BY cantidad_compras DESC;
"""

db.get_session().execute(text(vista_clientes_frecuentes))
print("✅ Vista 'vista_clientes_frecuentes' creada correctamente")


✅ Vista 'vista_clientes_frecuentes' creada correctamente


In [18]:
df_frecuentes = db.run_query("SELECT * FROM vista_clientes_frecuentes LIMIT 10")
df_frecuentes


,customer_id,cliente,cantidad_compras,total_gastado
0,60994,Rex Fields,6,1056.00
1,38149,Terrence Gray,5,520.00
2,69146,Toby Chambers,5,954.00
3,94115,Blake Dalton,5,1848.00
4,75042,Jake Hines,5,1045.00
5,73360,Allison Davies,5,1615.00
6,33150,Brock Bates,5,522.00
7,60837,Trisha Clay,5,752.00
8,69707,Elijah Eaton,5,630.00
9,76478,Alex David,5,1280.00


### Vista: Productos con Descuento
Esta vista permite observar cuáles productos se vendieron con descuento, cuántas veces y con qué promedio de rebaja. Útil para analizar promociones efectivas.

In [19]:
vista_productos_con_descuento = """
CREATE OR REPLACE VIEW vista_productos_con_descuento AS
SELECT 
    p.product_id,
    p.product_name,
    COUNT(s.sale_id) AS ventas_con_descuento,
    SUM(s.total_price) AS total_recaudado,
    AVG(s.discount) AS descuento_promedio
FROM sales s
JOIN products p ON s.product_id = p.product_id
WHERE s.discount > 0
GROUP BY p.product_id, p.product_name
ORDER BY ventas_con_descuento DESC;
"""

db.get_session().execute(text(vista_productos_con_descuento))
print("✅ Vista 'vista_productos_con_descuento' creada correctamente")


✅ Vista 'vista_productos_con_descuento' creada correctamente


In [21]:
df_descuentos = db.run_query("SELECT * FROM vista_productos_con_descuento LIMIT 10")
df_descuentos["descuento_promedio"] = df_descuentos["descuento_promedio"].apply(lambda x: f"{x:.2%}")
df_descuentos


,product_id,product_name,ventas_con_descuento,total_recaudado,descuento_promedio
0,422,Garlic - Primerba; Paste,39,6916.00,15.38%
1,314,Salmon Steak - Cohoe 8 Oz,35,6474.00,14.29%
2,118,Garbag Bags - Black,33,5973.00,14.24%
3,174,Guinea Fowl,33,4884.00,16.06%
4,48,Pecan Raisin - Tarts,32,5104.00,15.00%
5,99,Mustard - Seed,32,5522.00,15.62%
6,59,Pastry - Butterscotch Baked,31,3914.00,15.48%
7,23,Crab - Imitation Flakes,31,5415.00,15.16%
8,94,V8 - Berry Blend,31,5971.00,13.87%
9,184,Hersey Shakes,31,5415.00,14.84%


### Vista: Clientes Recientes
Objetivo: Mostrar los últimos clientes que realizaron compras, ordenados por fecha de la última compra.

In [3]:
from sqlalchemy import text

vista_clientes_recientes = """
CREATE OR REPLACE VIEW vista_clientes_recientes AS
SELECT
  c.customer_id,
  CONCAT(c.first_name, ' ', c.last_name) AS cliente,
  MAX(s.sale_time) AS ultima_compra
FROM customers c
JOIN sales s ON c.customer_id = s.customer_id
GROUP BY c.customer_id, cliente
ORDER BY ultima_compra DESC;
"""

db.get_session().execute(text(vista_clientes_recientes))
print("✅ Vista 'vista_clientes_recientes' creada correctamente")

✅ Vista 'vista_clientes_recientes' creada correctamente


In [4]:
df_recientes = db.run_query("SELECT * FROM vista_clientes_recientes LIMIT 10")
df_recientes

,customer_id,cliente,ultima_compra
0,8986,Linda Cummings,59:59.6
1,18075,Eduardo Irwin,59:59.6
2,64936,Earl Waller,59:59.4
3,61064,Peggy Sanford,59:59.3
4,94027,Ellen Villa,59:59.2
5,24838,Alan Armstrong,59:59.2
6,17746,Rodolfo Leach,59:59.0
7,33645,Milton Stone,59:58.9
8,20976,Jamal Suarez,59:58.9
9,51070,Vicky Matthews,59:58.9


### Vista: Ventas por Dia por Semana
Objetivo: Visualizar los días de la semana con más ventas para detectar patrones de comportamiento (por ejemplo, promociones efectivas en fines de semana o caídas los lunes).

In [2]:
from sqlalchemy import text  # ✔️ Import correcto

vista_ventas_por_dia = """
CREATE OR REPLACE VIEW vista_ventas_por_dia_semana AS
SELECT
  DAYNAME(s.sale_time) AS dia_semana,
  COUNT(*) AS cantidad_ventas,
  SUM(s.total_price) AS total_recaudado
FROM sales s
WHERE s.sale_time IS NOT NULL
GROUP BY dia_semana
ORDER BY cantidad_ventas DESC;
"""

db.get_session().execute(text(vista_ventas_por_dia))
print("✅ Vista 'vista_ventas_por_dia_semana' creada correctamente")

✅ Vista 'vista_ventas_por_dia_semana' creada correctamente


In [7]:
df_dias = db.run_query("SELECT * FROM vista_ventas_por_dia_semana")
df_dias = df_dias[df_dias['dia_semana'].notnull()]
df_dias


,dia_semana,cantidad_ventas,total_recaudado
1,Friday,1305,208033.00
2,Tuesday,1300,206792.00
3,Monday,1271,199737.00
4,Thursday,1264,197358.00
5,Wednesday,1259,202361.00
6,Sunday,1246,201692.00
7,Saturday,1226,197523.00
